In [1]:
import sys
sys.path.append('../../')

import numpy as np
import tickle
import glob
import pickle

from tqdm import tqdm
from joblib import Parallel, delayed
from VISDataPoint import VISDataPointV3

In [2]:
materials = ['None',
            'rock',
            'leaf',
            'water',
            'wood',
            'plastic-bag',
            'ceramic',
            'metal',
            'dirt',
            'cloth',
            'plastic',
            'tile',
            'gravel',
            'paper',
            'drywall',
            'glass',
            'grass',
            'carpet']

In [3]:
def get_data(file_name):
    dataPoint: VISDataPointV3 = tickle.load(file_name)
    return dataPoint.wav, materials.index(dataPoint.material)

In [4]:
# Train Data

train_files = glob.glob('/scratch/vis_data_v3/train/*.tkl')

selection_prob = 0.3
X_train, y_train = [], []

for train_file in tqdm(train_files):
    if np.random.random() > selection_prob:
        continue
    dataPoint: VISDataPointV3 = tickle.load(train_file)
    X_train.append(dataPoint.wav)
    y_train.append(materials.index(dataPoint.material))

100%|██████████| 20120/20120 [05:39<00:00, 59.24it/s] 


In [5]:
# Test Data

test_files = glob.glob('/scratch/vis_data_v3/test/*.tkl')

X_test, y_test = [], []

for test_file in tqdm(test_files):
    if np.random.random() > selection_prob:
        continue
    dataPoint: VISDataPointV3 = tickle.load(test_file)
    X_test.append(dataPoint.wav)
    y_test.append(materials.index(dataPoint.material))

100%|██████████| 7036/7036 [01:54<00:00, 61.30it/s] 


In [6]:
len(X_train), len(X_test)

(6109, 2046)

In [7]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf')
classifier.fit(X_train, y_train)

SVC()

In [8]:
y_pred = classifier.predict(X_test)

In [9]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=materials))

              precision    recall  f1-score   support

        None       0.41      0.98      0.58       829
        rock       0.33      0.02      0.03       124
        leaf       0.00      0.00      0.00        85
       water       0.00      0.00      0.00        86
        wood       0.20      0.01      0.01       160
 plastic-bag       0.00      0.00      0.00        27
     ceramic       0.00      0.00      0.00        34
       metal       0.00      0.00      0.00       154
        dirt       0.00      0.00      0.00       130
       cloth       0.60      0.23      0.34       112
     plastic       0.00      0.00      0.00       137
        tile       0.00      0.00      0.00         8
      gravel       0.00      0.00      0.00        10
       paper       0.00      0.00      0.00        89
     drywall       0.00      0.00      0.00        19
       glass       0.00      0.00      0.00         9
       grass       0.00      0.00      0.00        20
      carpet       0.00    

/home2/dhruv.kapur/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/dhruv.kapur/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/dhruv.kapur/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [10]:
with open('svm.pkl', 'wb') as f:
    pickle.dump(classifier, f)